# Coursework Assignment: Bias in AI

Link to dataset used (given in 'Project Suggestions'): https://www.kaggle.com/kabure/german-credit-data-with-risk

In [152]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [127]:
# load the data
df = pd.read_csv("./data/german_credit_data.csv", index_col=0)

# make a copy of the original data
original_df = df.copy()

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 85.9+ KB


In [129]:
df.describe()

,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000
std,11.375469,0.653614,2822.736876,12.058814
min,19.000000,0.000000,250.000000,4.000000
25%,27.000000,2.000000,1365.500000,12.000000
50%,33.000000,2.000000,2319.500000,18.000000
75%,42.000000,2.000000,3972.250000,24.000000
max,75.000000,3.000000,18424.000000,72.000000


In [130]:
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [131]:
# create age group column
df['Age_Group'] = np.nan

df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Age_Group
0,67,male,2,own,NaN,little,1169,6,radio/TV,good,NaN
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad,NaN
2,49,male,1,own,little,NaN,2096,12,education,good,NaN
3,45,male,2,free,little,little,7882,42,furniture/equipment,good,NaN
4,53,male,2,free,little,little,4870,24,car,bad,NaN


In [132]:
for col in [df]:
    print(col)

     Age     Sex  Job Housing Saving accounts Checking account  Credit amount  \
0     67    male    2     own             NaN           little           1169   
1     22  female    2     own          little         moderate           5951   
2     49    male    1     own          little              NaN           2096   
3     45    male    2    free          little           little           7882   
4     53    male    2    free          little           little           4870   
..   ...     ...  ...     ...             ...              ...            ...   
995   31  female    1     own          little              NaN           1736   
996   40    male    3     own          little           little           3857   
997   38    male    2     own          little              NaN            804   
998   23    male    2    free          little           little           1845   
999   27    male    2     own        moderate         moderate           4576   

     Duration              

In [133]:
# populate age group column
for col in [df]:
    col.loc[(col['Age'] > 18) & (col['Age'] <= 29), 'Age_Group'] = 'Young'
    col.loc[(col['Age'] > 29) & (col['Age'] <= 40), 'Age_Group'] = 'Young Adults'
    col.loc[(col['Age'] > 40) & (col['Age'] <= 55), 'Age_Group'] = 'Senior'
    col.loc[col['Age'] > 55, 'Age_Group'] = 'Elder' 
    
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Age_Group
0,67,male,2,own,NaN,little,1169,6,radio/TV,good,Elder
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad,Young
2,49,male,1,own,little,NaN,2096,12,education,good,Senior
3,45,male,2,free,little,little,7882,42,furniture/equipment,good,Senior
4,53,male,2,free,little,little,4870,24,car,bad,Senior


In [134]:
df['Sex'].value_counts()

male      690
female    310
Name: Sex, dtype: int64

In [135]:
df['Age_Group'].value_counts()

Young           371
Young Adults    355
Senior          203
Elder            71
Name: Age_Group, dtype: int64

In [136]:
def print_stats(df):
    print('mean: ', [round(num, 1) for num in list(df.mean())[1:]])
    print('variance: ', [round(num, 1) for num in list(df.var())[1:]])
    print('\n')
    zips = []
    for col in ['Housing', 'Saving accounts', 'Checking account',
       'Purpose', 'Risk']:
       vals = df[col].value_counts().values
       inds = df[col].value_counts().index
       z = list(zip(inds, vals))[:3]
       if col == 'Risk':
           z.append('-')
       zips.append(z)
       print(z)
       print('\n')
    print(list(zip(zips[0], zips[1], zips[2], zips[3], zips[4])))

In [137]:
df_male = df[df['Sex'] == 'male']
print_stats(df_male)

mean:  [1.9, 3448.0, 21.6]
variance:  [0.4, 8412806.3, 154.7]


[('own', 517), ('free', 89), ('rent', 84)]


[('little', 409), ('moderate', 71), ('quite rich', 47)]


[('little', 186), ('moderate', 183), ('rich', 43)]


[('car', 243), ('radio/TV', 195), ('furniture/equipment', 107)]


[('good', 499), ('bad', 191), '-']


[(('own', 517), ('little', 409), ('little', 186), ('car', 243), ('good', 499)), (('free', 89), ('moderate', 71), ('moderate', 183), ('radio/TV', 195), ('bad', 191)), (('rent', 84), ('quite rich', 47), ('rich', 43), ('furniture/equipment', 107), '-')]


In [138]:
df_female = df[df['Sex'] == 'female']
print_stats(df_female)

mean:  [1.8, 2877.8, 19.4]
variance:  [0.5, 6776346.3, 122.1]


[('own', 196), ('rent', 95), ('free', 19)]


[('little', 194), ('moderate', 32), ('rich', 19)]


[('little', 88), ('moderate', 86), ('rich', 20)]


[('car', 94), ('radio/TV', 85), ('furniture/equipment', 74)]


[('good', 201), ('bad', 109), '-']


[(('own', 196), ('little', 194), ('little', 88), ('car', 94), ('good', 201)), (('rent', 95), ('moderate', 32), ('moderate', 86), ('radio/TV', 85), ('bad', 109)), (('free', 19), ('rich', 19), ('rich', 20), ('furniture/equipment', 74), '-')]


In [139]:
df_Young = df[df['Age_Group'] == 'Young']
print_stats(df_Young)

mean:  [1.8, 3089.0, 20.8]
variance:  [0.3, 7261837.7, 142.6]


[('own', 248), ('rent', 113), ('free', 10)]


[('little', 242), ('moderate', 42), ('quite rich', 19)]


[('little', 115), ('moderate', 112), ('rich', 24)]


[('radio/TV', 117), ('car', 102), ('furniture/equipment', 84)]


[('good', 234), ('bad', 137), '-']


[(('own', 248), ('little', 242), ('little', 115), ('radio/TV', 117), ('good', 234)), (('rent', 113), ('moderate', 42), ('moderate', 112), ('car', 102), ('bad', 137)), (('free', 10), ('quite rich', 19), ('rich', 24), ('furniture/equipment', 84), '-')]


In [140]:
df_YoungAdults = df[df['Age_Group'] == 'Young Adults']
print_stats(df_YoungAdults)

mean:  [2.0, 3375.5, 21.5]
variance:  [0.4, 7646336.1, 139.2]


[('own', 278), ('free', 39), ('rent', 38)]


[('little', 201), ('moderate', 41), ('quite rich', 24)]


[('moderate', 100), ('little', 81), ('rich', 18)]


[('car', 128), ('radio/TV', 93), ('furniture/equipment', 58)]


[('good', 264), ('bad', 91), '-']


[(('own', 278), ('little', 201), ('moderate', 100), ('car', 128), ('good', 264)), (('free', 39), ('moderate', 41), ('little', 81), ('radio/TV', 93), ('bad', 91)), (('rent', 38), ('quite rich', 24), ('rich', 18), ('furniture/equipment', 58), '-')]


In [141]:
df_Senior = df[df['Age_Group'] == 'Senior']
print_stats(df_Senior)

mean:  [1.9, 3366.4, 20.2]
variance:  [0.4, 7986564.4, 146.1]


[('own', 143), ('free', 40), ('rent', 20)]


[('little', 117), ('moderate', 16), ('quite rich', 15)]


[('little', 57), ('moderate', 39), ('rich', 15)]


[('car', 79), ('radio/TV', 51), ('furniture/equipment', 36)]


[('good', 150), ('bad', 53), '-']


[(('own', 143), ('little', 117), ('little', 57), ('car', 79), ('good', 150)), (('free', 40), ('moderate', 16), ('moderate', 39), ('radio/TV', 51), ('bad', 53)), (('rent', 20), ('quite rich', 15), ('rich', 15), ('furniture/equipment', 36), '-')]


In [142]:
df_Elder = df[df['Age_Group'] == 'Elder']
print_stats(df_Elder)

mean:  [1.8, 3430.4, 20.5]
variance:  [0.7, 13329819.2, 192.5]


[('own', 44), ('free', 19), ('rent', 8)]


[('little', 43), ('rich', 5), ('quite rich', 5)]


[('little', 21), ('moderate', 18), ('rich', 6)]


[('car', 28), ('radio/TV', 19), ('business', 9)]


[('good', 52), ('bad', 19), '-']


[(('own', 44), ('little', 43), ('little', 21), ('car', 28), ('good', 52)), (('free', 19), ('rich', 5), ('moderate', 18), ('radio/TV', 19), ('bad', 19)), (('rent', 8), ('quite rich', 5), ('rich', 6), ('business', 9), '-')]


In [143]:
for group in [df_male, df_female, df_Young, df_YoungAdults, df_Senior, df_Elder]:
    print(round(sum(group['Risk'] == 'bad')/len(group) * 100, 1))

27.7
35.2
36.9
25.6
26.1
26.8
